# Programa auxiliar para separar las imágenes para Entrenamiento y Validación:

1) Cargar librerías:

In [ ]:
#@title Cargar Librerías
import os
import os.path
import numpy as np
import shutil

import zipfile

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = '/content/gdrive/MyDrive/GEMIS/objDetectionCursogramas' #@param {type:"string"}
auxiOrigen = path + '/Cursogramas/Generados'
#auxiOrigen = auxiPath + '/'
dirDestTrain = path +'/Cursogramas/train'
dirDestTest = path +'/Cursogramas/validation' 

2') Descomprime ZIP donde están los casos para distribuir (opcional):

In [ ]:
# define parámetros
zipFile =  '' #@param {type:"string"}
destinoFiles = auxiPath

# realiza unzip
zip_ref = zipfile.ZipFile(auxiPath +'/'+zipFile, 'r')
zip_ref.extractall(destinoFiles)
zip_ref.close()

# fuerza la actualizacion del drive
drive.flush_and_unmount()

In [ ]:
#@title Chequear archivos descomprimidos
# vuelve a montar
drive.mount('/content/gdrive', force_remount=True)

# indica cantidad de archivos extraidos
filesExtraidos = os.listdir(auxiOrigen)
print("> Archivos extraidos: ", len(filesExtraidos))

3) Levantar y procesar la carpeta de imágenes a distribuir:

In [ ]:
#@title Determinar archivos a distribuir

# crea los directorios, si es necesario
if not os.path.isdir(dirDestTrain):
  os.makedirs(dirDestTrain)
if not os.path.isdir(dirDestTest):
  os.makedirs(dirDestTest)

# variables auxiliares
all_images_array = []
allFileNames = []
train_FileNames = []
test_FileNames = []

# levanta la lista de imágenes de cada letra
allFileNames = [ fn for fn in os.listdir( auxiOrigen ) if fn.endswith('.png') or fn.endswith('.jpg') ]

# desordena y separa datos para train y para validación
np.random.shuffle(allFileNames)
train_FileNames, test_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames)*0.90)])

print('Total Imágenes Disponibles: ', len(allFileNames))
print('- Imágenes para Entrenamiento: ', len(train_FileNames))
print('- Imágenes para Validación: ', len(test_FileNames))
print("\n")

In [ ]:
#@title Mover archivos para Entrenamiento
cant = 0
cantXML = 0

# crea destinos
auxImgDir = dirDestTrain + '/images'
auxXMLDir = dirDestTrain + '/annotations'

if not os.path.isdir(auxImgDir):
  os.makedirs(auxImgDir)

if not os.path.isdir(auxXMLDir):
  os.makedirs(auxXMLDir)

for name in train_FileNames:  
  nameXML = name.replace(".png", ".xml")
  if os.path.isfile(auxiOrigen + '/' + nameXML):
    # mueve la imagen
    if os.path.isfile(auxImgDir + '/' + name):
      print("**", name, " ya existe en ", auxImgDir)      
    else:
      shutil.move(auxiOrigen + '/' + name, auxImgDir)
      cant = cant + 1
    # mueve el xml    
    if os.path.isfile(auxXMLDir + '/' + nameXML):
      print("**", nameXML, " ya existe en ", auxXMLDir)      
    else:
        shutil.move(auxiOrigen + '/' + nameXML, auxXMLDir)
        cantXML = cantXML + 1
  else:
    print("**", nameXML, " NO existe en ", auxiOrigen, " - se descarta imagen y XML!")      
print(cant, " imágenes movidas a ", auxImgDir)
print(cantXML, " XML movidos a ", auxXMLDir)
print("\n")

In [ ]:
#@title Mover archivos para Validación
cant = 0
cantXML = 0

# crea destinos
auxImgDir = dirDestTest + '/images'
auxXMLDir = dirDestTest + '/annotations'

if not os.path.isdir(auxImgDir):
  os.makedirs(auxImgDir)

if not os.path.isdir(auxXMLDir):
  os.makedirs(auxXMLDir)

for name in test_FileNames:
  nameXML = name.replace(".png", ".xml")
  if os.path.isfile(auxiOrigen + '/' + nameXML):
    # mueve la imagen
    if os.path.isfile(auxImgDir + '/' + name):
      print("**", name, " ya existe en ", auxImgDir)      
    else:
      shutil.move(auxiOrigen + '/' + name, auxImgDir)
      cant = cant + 1
    # mueve el xml    
    if os.path.isfile(auxXMLDir + '/' + nameXML):
      print("**", nameXML, " ya existe en ", auxXMLDir)      
    else:      
        shutil.move(auxiOrigen + '/' + nameXML, auxXMLDir)
        cantXML = cantXML + 1
  else:
    print("**", nameXML, " NO existe en ", auxiOrigen, " - se descarta imagen y XML!")      
print(cant, " imágenes movidas a ", auxImgDir)
print(cantXML, " XML movidos a ", auxXMLDir)
print("\n")

In [ ]:
#@title Fuerza la actualizacion del Drive

# actualiza y desmonta
drive.flush_and_unmount()

# vuelve a montar
drive.mount('/content/gdrive', force_remount=True)

print("Actualización terminada.")

4) Chequea que se haya pasado todo:

In [ ]:
#@title Controlar archivos huérfanos

# función auxiliar
def revisarArchivosHuerfanos(dir1, dir1Imagenes, dir2, dirSobra):
  cantOK = 0
  canError = 0
  for name1 in os.listdir( dir1 ):
    if dir1Imagenes: 
      name2 = name1.replace(".png", ".xml")
    else:
      name2 = name1.replace(".xml", ".png")
    if os.path.isfile(dir2 + '/' + name2):
      cantOK = cantOK + 1
    else:
      canError = canError + 1
      if dirSobra!='':
        shutil.move(dir1 + '/' + name1, dirSobra)
  print("\n> Resultados de controlar ", dir1, " contra ", dir2, ": ")
  print(" --- ", cantOK, " archivos OK.")
  print(" --- ", canError, " archivos Huérfanos movidos.")
  return cantOK, canError

# controla archivos de Entrenamiento
revisarArchivosHuerfanos(dirDestTrain + '/images', True, dirDestTrain + '/annotations', auxiOrigen)
revisarArchivosHuerfanos(dirDestTrain + '/annotations', False, dirDestTrain + '/images', auxiOrigen)

# controla archivos de Validación
revisarArchivosHuerfanos(dirDestTest + '/images', True, dirDestTest + '/annotations', auxiOrigen)
revisarArchivosHuerfanos(dirDestTest + '/annotations', False, dirDestTest + '/images', auxiOrigen)

print("\n")

In [17]:
#@title Mostrar Cantidades

# variables auxiliares
allFileNames = []
train_FileNames = []
test_FileNames = []

 # muestra cantidades de imágenes
allFileNames = [ fn for fn in os.listdir( auxiOrigen ) if fn.endswith('.png') or fn.endswith('.jpg')  ]
train_FileNames = [ fn for fn in os.listdir( dirDestTrain + '/images' ) if fn.endswith('.png') or fn.endswith('.jpg')  ]
train_da_FileNames = [ fn for fn in os.listdir( dirDestTrain + '/images' ) if fn.startswith('da') and (fn.endswith('.png') or fn.endswith('.jpg'))  ]
test_FileNames = [ fn for fn in os.listdir( dirDestTest + '/images' ) if fn.endswith('.png') or fn.endswith('.jpg') ]
test_da_FileNames = [ fn for fn in os.listdir( dirDestTest + '/images' ) if fn.startswith('da') and (fn.endswith('.png') or fn.endswith('.jpg'))  ]
print('Total de Imágenes en ', auxiOrigen,' : ', len(allFileNames))
print('\n- Total de Imágenes en Entrenamiento: ', len(train_FileNames))
print('--- Total de Imágenes sin DA en Entrenamiento: ', len(train_FileNames)-len(train_da_FileNames))
print('--- Total de Imágenes con DA en Entrenamiento: ', len(train_da_FileNames))
print('\n- Total de Imágenes en Validación: ', len(test_FileNames))
print('--- Total de Imágenes sin DA en Validación: ', len(test_FileNames)-len(test_da_FileNames))
print('--- Total de Imágenes con DA en Validación: ', len(test_da_FileNames))
print("\n\n")

# muestra cantidades de XML
allFileNames = [ fn for fn in os.listdir( auxiOrigen ) if fn.endswith('.xml') ]
train_FileNames = [ fn for fn in os.listdir( dirDestTrain + '/annotations') if fn.endswith('.xml') ]
train_da_FileNames = [ fn for fn in os.listdir( dirDestTrain + '/annotations') if fn.startswith('da') and fn.endswith('.xml') ]
test_FileNames = [ fn for fn in os.listdir( dirDestTest + '/annotations') if fn.endswith('.xml') ]
test_da_FileNames = [ fn for fn in os.listdir( dirDestTest + '/annotations') if fn.startswith('da') and fn.endswith('.xml') ]
print('Total de XML en ', auxiOrigen,' : ', len(allFileNames))
print('\n- Total de XML en Entrenamiento: ', len(train_FileNames))
print('--- Total de XML sin DA en Entrenamiento: ', len(train_FileNames)-len(train_da_FileNames))
print('--- Total de XML con DA en Entrenamiento: ', len(train_da_FileNames))
print('\n- Total de XML en Validación: ', len(test_FileNames))
print('--- Total de XML sin DA en Validación: ', len(test_FileNames)-len(test_da_FileNames))
print('--- Total de XML con DA en Validación: ', len(test_da_FileNames))
print("\n")

Total de Imágenes en  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados  :  0

- Total de Imágenes en Entrenamiento:  5548
--- Total de Imágenes sin DA en Entrenamiento:  3276
--- Total de Imágenes con DA en Entrenamiento:  2272

- Total de Imágenes en Validación:  500
--- Total de Imágenes sin DA en Validación:  367
--- Total de Imágenes con DA en Validación:  133



Total de XML en  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados  :  0

- Total de XML en Entrenamiento:  5548
--- Total de XML sin DA en Entrenamiento:  3276
--- Total de XML con DA en Entrenamiento:  2272

- Total de XML en Validación:  500
--- Total de XML sin DA en Validación:  367
--- Total de XML con DA en Validación:  133


